In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('aiddata-countries-only.csv')
df.head()

,Unnamed: 0,aiddata_id,aiddata_2_id,year,donor,recipient,commitment_amount_usd_constant,coalesced_purpose_code,coalesced_purpose_name
0,1,2414478.0,NaN,1977,Saudi Arabia,India,348718518.0,23030,Power generation/renewable sources
1,2,2414509.0,NaN,1977,Saudi Arabia,Brazil,191647004.0,23040,Electrical transmission/ distribution
2,3,2414635.0,NaN,1983,Saudi Arabia,India,79371799.0,21030,Rail transport
3,4,2414665.0,NaN,1984,Saudi Arabia,Taiwan,212202942.0,21030,Rail transport
4,5,2414667.0,NaN,1984,Saudi Arabia,Korea,134511154.0,21040,Water transport


In [3]:
# I have decided to drop sectors (for now...) because the categoy labels are too low-level and there are >80000 unspecified sectors
df['coalesced_purpose_name'].value_counts()

Sectors not specified                                                                          8076
Social/ welfare services                                                                       6228
Strengthening civil society                                                                    5600
Higher education                                                                               4593
Multisector aid                                                                                3026
Material relief assistance and services                                                        2243
Education facilities and training                                                              2144
Administrative costs                                                                           1866
Agricultural development                                                                       1654
Vocational training                                                                            1613


In [4]:
# Sanity check: Each row represents a unique year, donor, recipient and sector
df[(df['donor'] == 'United States') &(df['year'] == 1977)]

,Unnamed: 0,aiddata_id,aiddata_2_id,year,donor,recipient,commitment_amount_usd_constant,coalesced_purpose_code,coalesced_purpose_name
62322,62323,84788704.0,6613119.0,1977,United States,Korea,124945894.0,52010,Food security programmes/food aid
62732,62733,84985067.0,6520467.0,1977,United States,Thailand,720668.0,60010,Action relating to debt
63336,63337,85304884.0,6529786.0,1977,United States,Korea,6125682.0,60010,Action relating to debt
63958,63959,85627686.0,6519458.0,1977,United States,Thailand,9008356.0,31181,Agricultural education/training
64894,64895,86129314.0,6539424.0,1977,United States,Thailand,13512534.0,43040,Rural development
64922,64923,86142340.0,6529342.0,1977,United States,Korea,5855431.0,60010,Action relating to debt
65288,65289,86338964.0,6529675.0,1977,United States,Chile,180167.0,60010,Action relating to debt
65776,65777,86632775.0,6520060.0,1977,United States,Korea,1441337.0,60010,Action relating to debt
66560,66561,87035484.0,6520430.0,1977,United States,Korea,9879163.0,60010,Action relating to debt
67222,67223,87377778.0,6520134.0,1977,United States,Korea,360334.0,60010,Action relating to debt


In [5]:
#Group by year, donor, recipient and sum up the commitment amount
df = df[['year', 'donor', 'recipient', 'commitment_amount_usd_constant']]
df_new = df.groupby(['year', 'donor', 'recipient']).sum().reset_index()
df_new = df_new.reset_index().rename(columns={'index':'id', 'commitment_amount_usd_constant': 'amount'})

#Convert year from int to string
df_new['year'] = df_new['year'].astype(str)
df_new.head()

,id,year,donor,recipient,amount
0,0,1973,Australia,India,36565064.0
1,1,1973,Australia,Korea,1162012.0
2,2,1973,Australia,Thailand,8558787.0
3,3,1973,Belgium,Colombia,6541889.0
4,4,1973,Belgium,India,32709447.0


In [6]:
df_new.year.unique()

array(['1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980',
       '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988',
       '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
       '2013'], dtype=object)

In [7]:
def flatten_multi_index(df):
    mi = df.columns
    suffixes, prefixes = mi.levels
    col_names = [prefixes[i_p] + '_' + suffixes[i_s] for (i_s, i_p) in zip(*mi.labels)]
    df.columns = col_names
    return df

In [45]:
df_new.recipient.unique()

array(['India', 'Korea', 'Thailand', 'Colombia', 'Brazil', 'Chile',
       'Cyprus', 'Kuwait', 'Saudi Arabia', 'United Arab Emirates',
       'Taiwan', 'Australia', 'Austria', 'Belgium', 'Canada', 'Denmark',
       'France', 'Greece', 'Iceland', 'Ireland', 'Italy', 'Japan',
       'New Zealand', 'Norway', 'Poland', 'Portugal', 'Romania', 'Spain',
       'Sweden', 'United Kingdom', 'Liechtenstein', 'Finland', 'Hungary',
       'South Africa', 'Qatar', 'Czech Republic', 'Latvia', 'Estonia',
       'Lithuania', 'Slovenia', 'Slovak Republic', 'Luxembourg',
       'Switzerland', 'United States', 'Germany'], dtype=object)

## Data Processing - Global Distribution of Donors/Recipients

In [62]:
class DataProcessing_GlobalDistribution():
    
    def __init__(self):
        
        """
        Entities could be: donor/recipient
        This class contains functions to calculate the total sum of commitment amount as either a donor/recipient on a yearly basis
        """

    def final(self, df):

        # [DESTINATION]
        destination_count = self.global_stats(df, 'recipient')

        # [SOURCE]
        source_count = self.global_stats(df, 'donor')

        # Global count
        df = pd.concat([source_count, destination_count])
        all_country_PT = self.country_count_PT(df)
        
        # Sum
        net_all = pd.DataFrame()
        for country in list(set(list(df_new.donor.unique()) + list(df_new.recipient.unique()))):
            tmp = all_country_PT[all_country_PT['country'] == country]
            donor = tmp[tmp['category'] == 'donor'].drop(['country', 'category'], axis=1).reset_index(drop=True)
            recipient = tmp[tmp['category'] == 'recipient'].drop(['country', 'category'], axis=1).reset_index(drop=True)
            if((len(donor)>0) & (len(recipient)>0)):
                net = donor.add(recipient)
            elif (len(donor)==0):
                net = recipient 
            elif (len(recipient)==0):
                net = donor 
            net['country'] = country
            net_all = pd.concat([net_all, net])

        net_all['category'] = 'net'
        net_all = net_all.fillna(0).reset_index(drop=True)
        all_country_PT = pd.concat([all_country_PT, net_all])
        
        # Global density (in %)
        all_country_pctPT = self.country_pct_PT(df)

        return all_country_PT, all_country_pctPT
    
    def global_stats(self, df, category):

        country_stats = pd.DataFrame()

        tmp_g1 = pd.DataFrame(df.groupby([category])['amount'].sum()).reset_index().rename(columns={'amount': 'sum'}) 
        tmp_g1['year'] = "All"
        tmp_pcts = pd.DataFrame(df.groupby([category])['amount'].sum()).apply(lambda x: 100 * x / float(x.sum())).reset_index().rename(columns={'amount': 'pct'})
        tmp_g1['year'] = "All"
        new = pd.merge(tmp_g1, tmp_pcts, on=category, how='left')
        country_stats = pd.concat([country_stats, new])
        
        # segment by year
        for i in df['year'].unique():
            tmp = df[df['year'] == i]
            if(len(tmp) > 0):
                tmp_g1 = pd.DataFrame(tmp.groupby([category])['amount'].sum()).reset_index().rename(columns={'amount': 'sum'})     
                tmp_g1['year'] = int(i)
                tmp_pcts =  pd.DataFrame(tmp.groupby([category])['amount'].sum()).apply(lambda x: 100 * x / float(x.sum())).reset_index().rename(columns={'amount': 'pct'})
                tmp_pcts['year'] = int(i)
                tmp_pcts = pd.merge(tmp_g1, tmp_pcts, on=[category, 'year'], how='left')
                country_stats = pd.concat([country_stats, tmp_pcts])
                
        country_stats['category'] = category
        country_stats = country_stats.rename(columns={category: 'country'})
        return country_stats

    def country_count_PT(self, df):
        
        all_stats_PT = pd.pivot_table(df, values='sum', index=['country', 'category'], columns=['year']) 
        all_stats_PT.fillna(0, inplace=True)
        return all_stats_PT.reset_index()

    def country_pct_PT(self, df):

        all_stats_PT = pd.pivot_table(df, values='pct', index=['country', 'category'], columns=['year']) 
        all_stats_PT.fillna(0, inplace=True)
        return all_stats_PT.reset_index()

In [63]:
gd = DataProcessing_GlobalDistribution()

In [64]:
all_country_PT, all_country_pctPT = gd.final(df_new)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [55]:
all_country_PT[all_country_PT['country'] == 'Australia']

,country,category,1973,1974,1975,1976,1977,1978,1979,1980,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,All
0,Australia,donor,46285863.0,26950877.0,77691781.0,27718070.0,72648152.0,47483698.0,9763598.0,23283790.0,...,28927690.0,27058214.0,13864351.0,18754564.0,17680084.0,32806019.0,41545539.0,55649016.0,67265056.0,2.446616e+09
1,Australia,recipient,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54502260.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.572618e+09
42,Australia,net,46285863.0,26950877.0,77691781.0,27718070.0,72648152.0,47483698.0,9763598.0,77786050.0,...,28927690.0,27058214.0,13864351.0,18754564.0,17680084.0,32806019.0,41545539.0,55649016.0,67265056.0,5.019235e+09


In [56]:
all_country_PT[all_country_PT['country'] == "India"]

,country,category,1973,1974,1975,1976,1977,1978,1979,1980,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,All
34,India,recipient,2.285257e+09,3.697747e+09,3.000656e+09,2.233586e+09,3.197135e+09,2.927325e+09,1.622749e+09,2.968245e+09,...,3.340561e+09,4.144135e+09,5.386225e+09,5.044560e+09,3.706828e+09,3.826640e+09,3.652446e+09,4.970366e+09,4.550646e+09,1.339805e+11
0,India,net,2.285257e+09,3.697747e+09,3.000656e+09,2.233586e+09,3.197135e+09,2.927325e+09,1.622749e+09,2.968245e+09,...,3.340561e+09,4.144135e+09,5.386225e+09,5.044560e+09,3.706828e+09,3.826640e+09,3.652446e+09,4.970366e+09,4.550646e+09,1.339805e+11


In [37]:
max(all_country_PT.drop(['country', 'category'], axis=1).max(axis = 0).values)

133980469969.0

In [58]:
all_country_pctPT

year,country,category,1973,1974,1975,1976,1977,1978,1979,1980,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,All
0,Australia,donor,0.961191,0.396020,1.827483,0.749985,1.474190,0.989467,0.294701,0.210052,...,0.420308,0.354533,0.169810,0.204451,0.214898,0.427884,0.627657,0.568688,0.819761,0.667009
1,Australia,recipient,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.491686,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.701360
2,Austria,donor,0.000000,0.354515,0.000000,0.351258,0.052937,0.563842,0.074872,0.024543,...,0.259653,0.104863,0.141470,0.075755,0.095680,0.107271,0.089815,0.095747,0.087369,0.213757
3,Austria,recipient,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.422061,...,0.001853,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.209994
4,Belgium,donor,0.815109,0.640497,0.914151,0.977500,0.727554,0.670760,0.929703,0.000000,...,0.701566,0.765959,0.406474,0.578422,0.520336,0.540584,0.722586,0.365520,0.832586,0.584629
5,Belgium,recipient,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.333979,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050257
6,Brazil,donor,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000623,0.001841,0.000000,0.011930,0.000000,0.000000,0.000000,0.000309
7,Brazil,recipient,6.480674,4.505236,2.369291,2.261488,10.175290,3.157616,5.054619,3.586826,...,10.515045,6.200969,5.200444,6.278015,9.528164,13.815128,15.307284,24.967635,8.568072,9.980056
8,Canada,donor,9.094195,1.006580,2.188217,3.146457,0.790538,2.094562,2.421974,1.256355,...,0.967921,0.790199,0.767785,0.827036,0.900872,0.867918,0.752336,0.606226,0.667657,2.035511
9,Canada,recipient,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.796446,...,0.000000,0.000000,0.000000,0.000605,0.000000,0.000000,0.000000,0.000000,0.000000,1.109217


In [38]:
all_country_pctPT[["All"]].values.sum()

200.00000000000003

In [57]:
all_country_PT.to_csv('all_country_PT.csv', index=False)
all_country_pctPT.to_csv('all_country_pctPT.csv', index=False)

## Data Processing - Country-To-Country Connections

In [14]:
class DataProcessing_CountryToCountry():
    
    def __init__(self):
        
        """
        A country could be:
        1) Donor
        2) Recipient
        3) Both risk source and destination
        This is a class containing functions to find the density of connections a country has to other countries based on relationship on a yearly basis 
        """

    def final(self, df):
     
        """
        Output: Transformed data ready for input into chart
        
        Parameters:
        df: Cleaned dataframe of each relationship between country
        """
      
        wn_country_donor = self.country_to_country(df, ['donor','recipient'])                                                                      
        wn_country_recipient = self.country_to_country(df, ['recipient', 'donor'])   
        wn_country_net = self.net(df)
        wn_country_PT = self.country_flow_PT(wn_country_donor, wn_country_recipient, wn_country_net)
        return wn_country_PT
    
    def country_to_country(self, df, X):

        """
        Output: Count and percentage of connections to each country, segmented by year
        """
        
        country_stats = pd.DataFrame()

        tmp = pd.DataFrame(df.groupby(X)['amount'].sum()).reset_index()
        tmp['year'] = 'All'
        country_stats = pd.concat([country_stats, tmp])
        
        # segment by year
        for i in df['year'].unique():
            tmp = df[df['year'] == i]
            if(len(tmp) > 0):
                tmp = pd.DataFrame(tmp.groupby(X)['amount'].sum()).reset_index()    
                tmp['year'] = str(int(i))
                country_stats = pd.concat([country_stats, tmp])

        country_stats = country_stats[['year', 'donor', 'recipient', 'amount']]
        
        return country_stats

    def net(self, df):
        
        rsAll1= pd.DataFrame(df.groupby(['donor','recipient'])['amount'].sum()).reset_index()
        rsAll1 = rsAll1.rename(columns={'recipient': 'country 2', 'donor': 'country 1', 'amount': 'outflow'})
        rsAll1['year'] = 'All'
        rsAll2 = pd.DataFrame(df.groupby(['recipient', 'donor'])['amount'].sum()).reset_index()
        rsAll2 = rsAll2.rename(columns={'donor': 'country 2', 'recipient': 'country 1', 'amount': 'inflow'})
        rsAll2['year'] = 'All'
        
        rs1 = pd.DataFrame()
        rs2 = pd.DataFrame()
        for i in list(set(list(df.donor.unique()) + list(df.recipient.unique()))):
            tmp1 = df[(df['donor'] == i)]
            rs1 = pd.concat([rs1, tmp1])
            tmp2 = df[(df['recipient'] == i)]
            rs2 = pd.concat([rs2, tmp2])

        rs1 = rs1.rename(columns={'recipient': 'country 2', 'donor': 'country 1', 'amount': 'outflow'})
        rs2 = rs2.rename(columns={'donor': 'country 2', 'recipient': 'country 1', 'amount': 'inflow'})
    
        net = pd.merge(rs1, rs2, on=['country 1', 'country 2', 'year'], how='left')
        net = pd.concat([net, rsAll1, rsAll2])
        net['inflow'] = net['inflow'].fillna(0)
        net['outflow'] = net['outflow'].fillna(0)
        net['net'] = net['outflow'] - net['inflow']
        net['value'] = net['outflow'] + net['inflow']
        net = net.rename(columns={'country 1': 'country'})
        
        return net[['country', 'country 2', 'inflow', 'outflow', 'year', 'net', 'value']]
    
    def country_flow_PT(self, df1, df2, df3):

        df1_PT = pd.pivot_table(df1, values='amount', index=['donor'], columns=['year', 'recipient']) 
        df1_PT = flatten_multi_index(df1_PT).reset_index().rename(columns={'donor': 'country'})
        df1_PT['category'] = 'recipient'

        df2_PT = pd.pivot_table(df2, values='amount', index=['recipient'], columns=['year', 'donor']) 
        df2_PT = flatten_multi_index(df2_PT).reset_index().rename(columns={'recipient': 'country'})
        df2_PT['category'] = 'donor'

        df3_PT = pd.pivot_table(df3, values='net', index=['country'], columns=['year', 'country 2']) 
        df3_PT = flatten_multi_index(df3_PT).reset_index()
        df3_PT['category'] = 'net'
        
        df = pd.concat([df1_PT, df2_PT, df3_PT])
        df_PT = pd.pivot_table(df, index=['country'], columns=['category']) 
        df_PT = flatten_multi_index(df_PT).reset_index()
        df_PT = df_PT.fillna(0)

        return df_PT

In [15]:
ctc = DataProcessing_CountryToCountry()

In [16]:
wn_country_PT = ctc.final(df_new)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:73: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  after removing the cwd from sys.path.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:96: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [17]:
wn_country_PT.head()

,country,donor_Australia_1973,donor_Australia_1974,donor_Australia_1975,donor_Australia_1976,donor_Australia_1977,donor_Australia_1978,donor_Australia_1979,donor_Australia_1980,net_Australia_1980,...,net_United States_2008,recipient_United States_2008,donor_United States_2009,donor_United States_2010,donor_United States_2011,donor_United States_2012,donor_United States_2013,donor_United States_All,net_United States_All,recipient_United States_All
0,Australia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.572618e+09,-2.572618e+09,0.0
1,Austria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.699085e+08,-7.699085e+08,0.0
2,Belgium,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.843462e+08,-1.843462e+08,0.0
3,Brazil,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,55528517.0,31086679.0,33062881.0,21104514.0,49846811.0,1.354834e+10,-1.354834e+10,0.0
4,Canada,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.068597e+09,-4.068597e+09,0.0


In [42]:
max(wn_country_PT.drop(['country'], axis=1).max(axis = 0).values)

48830067295.0

In [18]:
wn_country_PT.to_csv('wn_country_PT.csv', index=False) 

In [19]:
class DataProcessing_CountrySnapshot():
    
    def __init__(self):
        
        """
        Country-level metrics
        """

    def countries_ranking(self, df_filt):
     
        """
        To rank countries connected to each country either as a donor/recipient based on commitment amount
        
        Parameters:
        df: Cleaned dataframe of each relationship between country
        """
    
        #[SOURCE]
        top_exposing = df_filt.groupby(['recipient', 'donor'])['amount'].sum().reset_index()
        top_exposing = top_exposing.sort_values('amount',ascending=False)
        top_exposing['category'] = 'exposed by'
        
        #[DESTINATION]
        top_exposedby = df_filt.groupby(['donor', 'recipient'])['amount'].sum().reset_index()
        top_exposedby = top_exposedby.sort_values('amount',ascending=False)
        top_exposedby['category'] = 'exposing'

        countries_ranking = pd.concat([top_exposing, top_exposedby])
        countries_ranking = countries_ranking.rename(columns={'amount': 'sum'})

        return countries_ranking
   

In [20]:
country = DataProcessing_CountrySnapshot()
countries_ranking = country.countries_ranking(df_new)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [21]:
countries_ranking.to_csv('countries_ranking.csv', index=False)